In [13]:
# !pip3 install gensim

  Using cached wrapt-1.16.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.3 MB/s eta 0:00:00
Using cached wrapt-1.16.0-cp310-cp310-macosx_11_0_arm64.whl (38 kB)


In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import json
import gensim.downloader as api


In [31]:
with open('Json Task1/train_processed.json', 'r') as f:
    task1_train_data = json.load(f)

with open('Json Task1/val_processed.json', 'r') as f:
    task1_val_data = json.load(f)

with open('Json Task1/test_processed.json', 'r') as f:
    task1_test_data = json.load(f)

with open('Json Task2/train_processed.json', 'r') as f:
    task2_train_data = json.load(f)

with open('Json Task2/val_processed.json', 'r') as f:
    task2_val_data = json.load(f)

with open('Json Task2/test_processed.json', 'r') as f:
    task2_test_data = json.load(f)

In [32]:
task1_train_data

{'0': {'text': 'therefore interpreting statutory provisions courts keep mind objectives purpose statute enacted',
  'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']},
 '1': {'text': 'petitioner w.p.no.15821 2008 never considered appointment national rural employment guarantee scheme either employment exchange sponsorship outsourcing agencies',
  'labels': ['O',
   'B_CASE_NUMBER',
   'I_CASE_NUMBER',
   'O',
   'O',
   'O',
   'B_ORG',
   'I_ORG',
   'I_ORG',
   'I_ORG',
   'I_ORG',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O']},
 '2': {'text': 'factum accident allegation rash negligent driving causing death sukendra pal singh denied',
  'labels': ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'B_OTHER_PERSON',
   'I_OTHER_PERSON',
   'I_OTHER_PERSON',
   'O']},
 '3': {'text': '.. 36 .. w.a.no.655/2012 others meaning thereby except interview commission entire procedure recruitment emergency appointment followed',
  'labels': ['O',
   'O',
   'O',
  

In [33]:
word2vec_model = api.load('word2vec-google-news-300')
vocab_size, embedding_dim = word2vec_model.vectors.shape

In [34]:
# word_to_index = {word: index for index, word in enumerate(word2vec_model.index_to_key)}
# input_word = "Nalish"

# # Check if the input word is in the vocabulary
# if input_word in word_to_index:
#     input_index = word_to_index[input_word]
#     input_tensor = torch.tensor([input_index])

#     # Pass the input tensor to the embedding layer
#     embedded_output = embedding_layer(input_tensor)
#     print(embedded_output)
# else:
#     print(f"The word '{input_word}' is not in the vocabulary.")

word2vec = torch.FloatTensor(word2vec_model.vectors)

In [82]:
class GRUModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_size, pretrained_embedding):
        super(GRUModel, self).__init__()
        self.embedding_layer = nn.Embedding.from_pretrained(pretrained_embedding, freeze=True)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):
        x = self.embedding_layer(x)
        out, _ = self.gru(x)
        out = self.fc(out[:, -1, :])  
        return out

class Task_data(Dataset):
    def __init__(self, data, embedding_type):
        self.data = data
        self.length = len(self.data)

        if embedding_type == "word2vec":
            embedding_layer = nn.Embedding.from_pretrained(word2vec, freeze=True)
            word_to_index = {word: index for index, word in enumerate(word2vec_model.index_to_key)}
            data_indices = []
            for id in data.keys():
                input_sequence = data[id]['text'].split(' ')
              
                # print(input_sequence)
                # sentence_indices = [word_to_index[word] for word in input_sequence if word in word_to_index]
                sentence_indices = [word_to_index[word] for word in input_sequence]

                data_indices.append(torch.tensor(sentence_indices))

            padded_input_tensors = nn.utils.rnn.pad_sequence(data_indices, batch_first=True)
            embedded_outputs = embedding_layer(padded_input_tensors)
            print(embedded_outputs.shape)
                # input_indices = [word_to_index[word] for word in input_sequence if word in word_to_index]
                

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        return self.data[index]    

In [83]:
task1_train_data[str(32)]

KeyError: '32'

In [84]:
d = Task_data(task1_train_data, embedding_type = "word2vec")

KeyError: 'w.p.no.15821'

In [11]:
# definig data_loaders
batch_size = 64

task1_train_dataloader = DataLoader(Task_data(task1_train_data), batch_size=batch_size, shuffle=True)
task1_val_dataloader =  DataLoader(Task_data(task1_val_data), batch_size=batch_size, shuffle=True)
task1_test_dataloader =  DataLoader(Task_data(task1_test_data), batch_size=batch_size, shuffle=False)

task2_train_dataloader =  DataLoader(Task_data(task2_train_data), batch_size=batch_size, shuffle=True)
task2_val_dataloader = DataLoader(Task_data(task2_val_data), batch_size=batch_size, shuffle=True)
task2_test_dataloader = DataLoader(Task_data(task2_test_data), batch_size=batch_size, shuffle=False)



In [ ]:
embedding_dim = 300  # Adjust based on the chosen pre-trained embeddings
hidden_dim = 128
output_size = num_classes  # Adjust based on your task
learning_rate = 0.001
num_epochs = 10

In [ ]:
for dataset, (word2vec_embedding, glove_embedding, fasttext_embedding) in zip([dataset1, dataset2], [word2vec, glove, fasttext]):
    for pretrained_embedding, embedding_name in zip([word2vec_embedding, glove_embedding, fasttext_embedding],
                                                   ['Word2Vec', 'GloVe', 'FastText']):
        model = GRUModel(embedding_dim, hidden_dim, output_size, pretrained_embedding)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        # Data processing (convert data to PyTorch tensors, create DataLoader, etc.)
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

        # Training loop
        for epoch in range(num_epochs):
            model.train()
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            # Validation
            model.eval()